# Notebook for testing pyimagej with TrackMate Scripts

This is a pyimagej TrackMate integration test that follows instruction [here](https://github.com/imagej/pyimagej/issues/246#issuecomment-1410871419).

Import Headers

In [1]:
import sys
import os
import imagej
import imagej.doctor
import scyjava as sj

## Mark the Local Working Directory
The local working path need to be stored as pyimagej will change the CWD after initialization

In [2]:
CWD = os.getcwd() 

## JAVA_HOME setting
Export `JAVA_HOME` path for the virtual environment if `JAVA_HOME` is not set and we are in a conda environment

In [3]:
if (not os.environ.get('JAVA_HOME')) and os.environ.get('CONDA_DEFAULT_ENV'):
    # infer the conda environment via sys.executable path
    os.environ['JAVA_HOME'] = os.sep.join(sys.executable.split(os.sep)[:-1] + ['Library'])

## pyimagej checkup

In [4]:
imagej.doctor.checkup()


Checking Python:
--> Python executable = c:\Users\kko\mambaforge\envs\pyimagej\python.exe

Checking environment:
--> CONDA_PREFIX = C:\Users\kko\mambaforge\envs\pyimagej
--> Python executable matches Conda environment.

Checking Python dependencies:
--> jgo: c:\Users\kko\mambaforge\envs\pyimagej\lib\site-packages\jgo\__init__.py
--> scyjava: c:\Users\kko\mambaforge\envs\pyimagej\lib\site-packages\scyjava\__init__.py
--> imglyb: c:\Users\kko\mambaforge\envs\pyimagej\lib\site-packages\imglyb\__init__.py
--> pyimagej: c:\Users\kko\mambaforge\envs\pyimagej\lib\site-packages\imagej\__init__.py

Checking Maven:
--> Maven executable = C:\Users\kko\mambaforge\envs\pyimagej\Library\bin\mvn.CMD
$ mvn -v
Apache Maven 3.8.7 (b89d5959fcde851dcb1c8946a785a163f14e1e29)
Maven home: C:\Users\kko\mambaforge\envs\pyimagej\Library
Java version: 1.8.0_332, vendor: Azul Systems, Inc., runtime: C:\Users\kko\mambaforge\envs\pyimagej\Library\jre
Default locale: en_GB, platform encoding: Cp1252
OS name: "windo

## Initialize pyimagej with custom Fiji app

In [5]:
FIJI_DIR = "C:/Users/kko/Software/Fiji.app"

print("Initializing Fiji on JVM...")
ij = imagej.init(FIJI_DIR,mode='headless') # choose headless mode if you are running on cloud
# ij = imagej.init(FIJI_DIR,mode='interactive') # to show image in Fiji you need to use interactive mode, 
print(ij.getApp().getInfo(True))

Initializing Fiji on JVM...
ImageJ2 2.9.0/1.53t; Java 1.8.0_332 [amd64]; 160MB of 7195MB


## Run external scripts
pyimagej allows loading of scripts without explicitly pasting inline. We have to call the script file path using the Java-side method [`ij.script().run(File, boolean, Object[])`](https://github.com/scijava/scijava-common/blob/scijava-common-2.90.1/src/main/java/org/scijava/script/ScriptService.java#L130-L148). 

The `ij.py.run_script` Python-side function is just a wrapper around some `ij.script().run(...)` methods. You can convert your Python arguments to an `Object[]` using `ij.py.jargs`. If you want to block until script execution is complete, call `get()` on the Future returned by `ij.script().run`.

#### Python side

In [6]:
# define python side arguments
args = {
    'name': 'JK',
    'age': 99,
    'city': 'London'
}

# load the script file
with open(os.path.join(CWD,'test.ijm'),"r") as f:
    script_ijm = f.read()

In [7]:
# Run with ij.py.run_script() function
result = ij.py.run_script('ijm', script_ijm, args)
print(result.getOutput("greeting1"))

Hello JK.


[java.lang.Enum.toString] [INFO] script:script.ijm = [[greeting1, greeting2], [Hello JK., You are 99 years old, and live in London.]]


#### Java side

In [8]:
File = sj.jimport('java.io.File')
jfile = File(os.path.join(CWD,'test.ijm'))

In [9]:
# convert python side stuffs to java side
jargs = ij.py.jargs(args)
result_future = ij.script().run(jfile,True,jargs)

# get the result from java future, blocking will occur here
result = ij.py.from_java(result_future.get().getOutput("greeting"))
print(result)

None


[java.lang.Enum.toString] [INFO] script:c:\Users\kko\Projects\Cell Tracking\pyimagej-trackmate\test.ijm = [[greeting1, greeting2], [Hello JK., You are 99 years old, and live in London.]]


### Jython Script

In thise section, we are attempting to load the image using python side image loader to mimic workflow in python image analysis, then parse the image to [`test.py`](./test.py) script.
Jython arguments can be parsed through two ways:

1. `sys.argv`
2. Jython specific way with `#@ String` etc

We are following the second approach which sticks more to the the pyimagej interface.

#### Image Loading

In [10]:
from skimage import io

im = io.imread('https://fiji.sc/samples/FakeTracks.tif')

In [11]:
im.shape

(50, 128, 128)

In [18]:
# convert the python size image to java side
imj = ij.py.to_java(im)
imp = ij.py.to_imageplus(im)

print(type(imj),type(imp))

<java class 'net.imglib2.img.ImgView'> <java class 'ij.ImagePlus'>


Here you can see `imj` is a ImageJ2 class object `net.imglib2.img.ImgView`, while imp is the ImageJ1 class object `ij.ImagePlus`

#### Python side

We first run with the native ImageJ1 class parsing.

In [13]:
# define python side arguments
args = {
    'imp': imp # java side ImagePlus object
}

# load the script file
with open(os.path.join(CWD,'test.py'),"r") as f:
    script_py = f.read()

# Run with ij.py.run_script() function
result = ij.py.run_script('py', script_py, args)
print(result.getOutput("foo"))
print(result.getOutput("bar"))
print(result.getOutput("shape"))

[java.lang.Enum.toString] [WARNING] Ignoring invalid parameter: output Type outputName will declare the variable of the specified name as an output parameter with the given type. 
[java.lang.Enum.toString] [WARNING] Ignoring invalid parameter: output Type outputName will declare the variable of the specified name as an output parameter with the given type. 


0
0.10000000149011612
[128, 128, 50, 1, 1]


[java.lang.Enum.toString] complete Fiji Python script
[java.lang.Enum.toString] [INFO] script:script.py = [[foo, bar], [0, 0.1]]
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: shape [[I]
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: shape [[I]


Everything is good except for the warning that output scripting parameter `shape` is not defining the type. 

Let's test the backward compatibility with ImageJ2 file parsing to Jython side `ImagePlus` parameter input

In [19]:
# define python side arguments
args = {
    'imp': imj # java side net.imglib2.img.ImgView object with jython input parameter type ImagePlus
}

# load the script file
with open(os.path.join(CWD,'test.py'),"r") as f:
    script_py = f.read()

# Run with ij.py.run_script() function
result = ij.py.run_script('py', script_py, args)
print(result.getOutput("foo"))
print(result.getOutput("bar"))
print(result.getOutput("shape"))

[java.lang.Enum.toString] [WARNING] Ignoring invalid parameter: output Type outputName will declare the variable of the specified name as an output parameter with the given type. 
[java.lang.Enum.toString] [WARNING] Ignoring invalid parameter: output Type outputName will declare the variable of the specified name as an output parameter with the given type. 
[java.lang.Enum.toString] [ERROR] For input imp: incompatible object net.imglib2.img.ImgView for type ij.ImagePlus


0
0.10000000149011612
[128, 128, 50, 1, 1]


[java.lang.Enum.toString] complete Fiji Python script
[java.lang.Enum.toString] [INFO] script:script.py = [[foo, bar], [0, 0.1]]
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: shape [[I]
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: shape [[I]


Backward compatiblity is proven.

### Java side

In [20]:
File = sj.jimport('java.io.File')
jfile = File(os.path.join(CWD,'test.py'))

# convert python side stuffs to java side
# define python side arguments
args = {
    'imp': imp # java side ImagePlus object
}
jargs = ij.py.jargs(args)
result_future = ij.script().run(jfile,True,jargs)

# get the result from java future, blocking will occur here
result = result_future.get()
print(ij.py.from_java(result.getOutput("foo")))
print(ij.py.from_java(result.getOutput("bar")))
print(ij.py.from_java(result.getOutput("shape")))

[java.lang.Enum.toString] [WARNING] Ignoring invalid parameter: output Type outputName will declare the variable of the specified name as an output parameter with the given type. 
[java.lang.Enum.toString] [WARNING] Ignoring invalid parameter: output Type outputName will declare the variable of the specified name as an output parameter with the given type. 


0
0.10000000149011612
[128 128  50   1   1]


[java.lang.Enum.toString] complete Fiji Python script
[java.lang.Enum.toString] [INFO] script:c:\Users\kko\Projects\Cell Tracking\pyimagej-trackmate\test.py = [[foo, bar], [0, 0.1]]
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: shape [[I]
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: shape [[I]


In [21]:
File = sj.jimport('java.io.File')
jfile = File(os.path.join(CWD,'test.py'))

# convert python side stuffs to java side
# define python side arguments
args = {
    'imp': imj # java side net.imglib2.img.ImgView object with jython input parameter type ImagePlus
}
jargs = ij.py.jargs(args)
result_future = ij.script().run(jfile,True,jargs)

# get the result from java future, blocking will occur here
result = result_future.get()
print(ij.py.from_java(result.getOutput("foo")))
print(ij.py.from_java(result.getOutput("bar")))
print(ij.py.from_java(result.getOutput("shape")))

[java.lang.Enum.toString] [WARNING] Ignoring invalid parameter: output Type outputName will declare the variable of the specified name as an output parameter with the given type. 
[java.lang.Enum.toString] [WARNING] Ignoring invalid parameter: output Type outputName will declare the variable of the specified name as an output parameter with the given type. 
[java.lang.Enum.toString] [ERROR] For input imp: incompatible object net.imglib2.img.ImgView for type ij.ImagePlus


0
0.10000000149011612
[128 128  50   1   1]


[java.lang.Enum.toString] complete Fiji Python script
[java.lang.Enum.toString] [INFO] script:c:\Users\kko\Projects\Cell Tracking\pyimagej-trackmate\test.py = [[foo, bar], [0, 0.1]]
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: shape [[I]
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: shape [[I]
